# Transfer Learning with TensorFlow Part 2: Fine Tuning

In the previous notebook, we covered transfer learning *feature extraction*, now it's time to learn about a new kind of transfer learning: *fine-tuning*

## Creating helper functions

In previous notebooks, we've created a bunch of helper functions; however, this is tedious.

It's a good idea to put functions you'll want to use again in a script you can download and import into your notebooks (or elsewhere)

In [6]:
# Import helper functions
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

## Let's get some data

This time we're going to see how we can use the pretrained models within `tf.keras.applications` and apply them to our own problem (recognizing images of food)

In [7]:
# Check out how many images and subdirectories are in our dataset
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test

In [3]:
# Create training and test directories path
train_dir="10_food_classes_10_percent/train/"
test_dir="10_food_classes_10_percent/test/"

**Note**: Rather than using the `ImageGenerator` function, we are going to use `tf.keras.preprocessing.image_dataset_from_directory`. This is due to its functionality as a TensorFlow datatype in addition to its speed attributes.

In [4]:
import tensorflow as tf
IMG_SIZE=(224,224)
BATCH_SIZE=32

train_data_10_percent=tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                          image_size=IMG_SIZE,
                                                                          label_mode="categorical",
                                                                          batch_size=BATCH_SIZE)

test_data=tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                              image_size=IMG_SIZE,
                                                              label_mode="categorical",
                                                              batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [5]:
# Let's look at the dataset
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [7]:
# Check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [8]:
# See an example of a batch of data
for images, labels in train_data_10_percent.take(1):
    print(images,labels)

tf.Tensor(
[[[[ 36.07143     35.07143     51.07143   ]
   [ 32.59694     31.59694     49.59694   ]
   [ 35.438774    32.438774    53.438774  ]
   ...
   [145.01532    113.87238    115.0051    ]
   [150.14804    114.387825   119.45925   ]
   [154.33145    114.97431    121.04574   ]]

  [[ 32.55102     31.55102     47.55102   ]
   [ 34.785713    33.785713    51.785713  ]
   [ 35.285713    32.285713    53.285713  ]
   ...
   [139.04083    108.48467    105.341835  ]
   [139.28572    105.28573    103.500015  ]
   [147.65833    111.658325   111.801186  ]]

  [[ 28.948978    27.948978    43.94898   ]
   [ 27.80102     26.80102     42.801018  ]
   [ 25.95408     22.95408     41.52551   ]
   ...
   [137.71428    107.974464    99.49998   ]
   [145.14275    112.78561    105.015175  ]
   [132.09183     99.73469     91.45402   ]]

  ...

  [[  2.6429005    9.071428     0.28061998]
   [  8.913334    17.000046     0.        ]
   [ 38.64763     49.6731      11.673163  ]
   ...
   [ 31.857056    13.979